In [6]:
prompt_meu_coding_bse_08 = """
# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {{"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}}
必要的额外信息: NaN

## 输出
<CODE>
import pandas a pd

def check_meu_4_1(df):
    '''
    代码的注释要清晰, 展示你分别验证subject, condition和constrain的过程.
    "subject": "上市公司大股东 | 董监高", 
    "condition": "计划通过本所集中竞价或大宗交易减持股份", 
    "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", 
    "contextual_info": NaN
    '''

    df['meu_4_2_subject'] = False  # 初始化subject列
    df['meu_4_2_condition'] = False  # 初始化condition列
    df['meu_4_2_constrain'] = False  # 初始化constrain列
    
    # 标记valid的subject
    is_major_shareholder = (
        df['股东身份'].isin(['控股股东', '实际控制人', '持股5%以上股东']) |
        (df['持股比例'] >= 0.05)
    )
    is_director = df['股东身份'].isin(['董监高'])
    valid_subject = is_major_shareholder | is_director

    # 标记valid的condition
    valid_condition = df['减持方式'].isin(['竞价交易', '大宗交易'])

    # 标记valid的constrain
    df['交易日差'] = (df['计划开始日'] - df['计划披露日']).dt.days
    has_plan = df['存在减持计划']
    valid_constraint = df['交易日差'] >= 15 & df['存在减持计划']
    
    # 标记各条件满足情况
    df.loc[valid_subject, 'meu_4_1_subject'] = True
    df.loc[valid_condition, 'meu_4_1_condition'] = True
    df.loc[valid_constraint, 'meu_4_1_constraint'] = True
    
    # 判断违规情况：满足主体和条件但不满足约束
    is_violation = (
        valid_subject & 
        valid_condition & 
        (~valid_constraint)
    )
    
    # 标记违规行
    df.loc[is_violation, 'meu_4_1_compliance'] = False
    
    return df

</CODE>
    
# 下面轮到你来完成这个任务
法律的最小可执行单元编号: {meu_id}
法律的最小可执行单元内容: {meu_content}
必要的额外信息: {note}


# 附加信息
## 你可以操作的dataframe的columns: 
['日期', '日收益率', '收盘价', '前收盘价', '上市日期', '发行价格', '净资产增长率', '净利润增长率', '净资产',
'净利润', '总股本', '每股净资产', '每股净利润', '公告类型', '公告日期', '复权因子', '收盘价减每股净资产',
'收盘价减每股净利润', '公司涉嫌证券期货违法犯罪事件', '持股比例', '持股数量', '股东身份', '减持方式', '拟减持原因',
'存在减持计划', '计划减持比例', '计划价格下限', '计划价格上限', '当日减持比例', '累计减持比例', '计划披露日',
'计划开始日', '计划结束日', '离任日期', '公司简称', '股东', '股东涉嫌证券期货违法犯罪事件']

## 其中: 
- 所有日期相关的列都已经是pd.datetime格式, ['日期']列本身就是交易日, 不需要额外计算交易日, 但是自然日需要计算. 
- 所有价格的单位都是元而非万元, 千元或百万元; 所有的比例都是小数而非百分数
- 这是一个二维的面板数据, 根据['日期', '公司简称', '股东']可以定位到特定公司的特定股东在特定交易日的状态, 以及公司在该日的状态. 如果你要分公司和股东地考虑一些合规要求, 需要考虑按照 ['公司简称', '股东']分组并在组内完成. 
- 交易方式有['竞价交易', '大宗交易', '融券卖出'], 竞价交易包括集中竞价. 
- ['公司/股东涉嫌证券期货违法犯罪事件'] 有['被中国证监会及其派出机构立案调查', '中国证监会及其派出机构立案调查结束', '行政处罚决定作出', '被司法机关立案侦查', '司法机关立案侦查结束', '被本所公开谴责']
- ['股东身份'] 有 ['董监高', '离任董监高', '控股股东', '实际控制人', '持股5%以上股东', '普通股东'], 这些标签不会并列出现, 因此有时候董监高/离任董监高持股在5%以上但并没有标明'持股5%以上股东', 你需要从['持股比例']手动筛选. 
- ['存在减持计划']是一个布尔值，表示该股东当天是否有减持计划
- 假设公司上市发行价就是最早的非空的收盘价, 公司发行时的控股股东和实际控制人就是第一个非空的收盘价的同一天的控股股东和实际控制人. 
- ['持股比例'] 全部是公开上市前取得的股份. 不考虑承诺和限制出售的股份. 

"""


In [7]:
meu_data = [
  {
    "MEU_id": "MEU_12_1",
    "subject": "公开发行股票并上市时的控股股东 | 公开发行股票并上市时的实际控制人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
  },
  {
    "MEU_id": "MEU_12_2",
    "subject": "上市公司控股股东 | 实际控制人 | 一致行动人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内，上市公司任一交易日股票收盘价低于最近一个会计年度或者最近一期财务会计报告期末每股归属于上市公司股东的净资产",
    "contextual_info": "股票收盘价以最近一个会计年度或者最近一期财务会计报告资产负债表日为基准分别向后复权计算"
  },
  {
    "MEU_id": "MEU_12_3",
    "subject": "上市公司控股股东 | 实际控制人 | 一致行动人",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：上市公司最近一期经审计的财务报告的归属于上市公司股东的净利润为负",
    "contextual_info": ""
  },
  {
    "MEU_id": "MEU_12_4",
    "subject": "控股股东 | 实际控制人（上市后不再具备相关主体身份的）",
    "condition": "计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
  },
  {
    "MEU_id": "MEU_12_5",
    "subject": "公开发行时持股5%以上的第一大股东 | 一致行动人",
    "condition": "上市公司在公开发行股票并上市时披露为无控股股东、实际控制人, 且计划通过集中竞价交易或大宗交易减持股份且首次披露减持计划, 且不存在已经按照本指引第四条披露减持计划，或者中国证监会另有规定的情况的",
    "constraint": "不得存在下列情形：最近20个交易日内任一交易日股票收盘价低于公开发行股票并上市的发行价格",
    "contextual_info": "股票收盘价以公开发行股票并上市之日为基准向后复权计算"
  }
]

In [8]:
# 填充数据
prompt_list = [
    prompt_meu_coding_bse_08.format(
        meu_id=item["MEU_id"],
        meu_content=f'{item["subject"]} | {item["condition"]} | {item["constraint"]} | {item["contextual_info"]}',
        note="无"
    )
    for item in meu_data
]

# 打印结果
for prompt in prompt_list:
    print(prompt)


# 身份
你是一个精通法律的程序员. 擅长编写代码以检验特定案例对于特定法律要求的合规性. 

## MEU概念简述
MEU (Minimum Executable Unit) 是法律条文拆解出的最小合规单元，包含：
- MEU_id: MEU的编号, 通常为"MEU_n_k", 其中n是其所属的法条的编号, k是其在法条内部的编号
- subject: 责任主体（如"控股股东") 
- condition: 触发条件（如"减持股份")  
- constraint: 约束内容（如"提前15日公告") 
- contextual_info: 补充说明（如价格计算方式）


# 任务
你现在面临一个编写代码以检验某法律事件的合规性的任务, 你需要编写一个函数, 这个函数的输入是一个日频的pandas dataframe, 每一行是在某个交易日内公司的情况和股东的情况, 这个函数会检验该dataframe的每一行在某一个给定的法律最小可执行单元上的合规性, 标注在特定的列上, 并返回标注后的dataframe. 


# 更多要求
 - 你可以进行任意长度的思考, 然后用<CODE></CODE>包裹你的代码. 
 - 你需要按照下面这个框架编写代码: 1. 标记valid的subject(meu_n_k_subject); 2. 标记valid的condition(meu_n_k_condition); 3. 标记valid的constrain(meu_n_k_constrain); 4. 在dataframe中记录subject, condition和constrain的valid情况; 5. 计算整体的违约情况并标记在dataframe上(meu_n_k_compliance). 

# example
下面是一个例子:

## 输入
法律的最小可执行单元编号: MEU_4_1
法律的最小可执行单元内容: {"subject": "上市公司大股东 | 董监高", "condition": "计划通过本所集中竞价或大宗交易减持股份", "constraint": "应当及时通知公司，并在首次卖出的15个交易日前向本所报告并预先披露减持计划", "contextual_info": NaN}
必要的额外信息: NaN

## 输出
<CODE>
import pan

In [9]:
import asyncio
from tqdm.asyncio import tqdm_asyncio
import pandas as pd
import os
from utils.call_gpt import call_gpt_async 

async def process_prompts_async(
    prompt_list,
    output_file_path,
    max_concurrency=5,
):
    """
    异步处理prompt列表并保存结果到Excel文件
    
    参数：
    prompt_list: 提示词字符串列表
    output_file_path: 输出文件路径
    max_concurrency: 最大并行请求数（默认5）
    """
    
    # 确保输出目录存在
    # (os.path.dirname(output_file_path), exist_ok=Trueos.makedirs)
    
    # 共享状态
    results = []
    failed_prompts = []
    semaphore = asyncio.Semaphore(max_concurrency)

    async def process_single_prompt(prompt):
        """处理单个提示词的异步任务"""
        async with semaphore:
            try:
                content, reasoning_content, api_usage = await call_gpt_async(
                    prompt=prompt,
                    api_key="35684824-1776-48b6-94fd-96c2e99d0724",
                    base_url="https://ark.cn-beijing.volces.com/api/v3",
                    # model="ep-20250217153824-9xcbx", # deepseek-r1 671b
                    model="ep-20250329012323-5qdxc", # deepseek-v3 250324
                    # temperature=0.6,
                )

                return {
                    "prompt": prompt,
                    "response": content,
                    "reasoning_content": reasoning_content,
                    "api_usage": api_usage
                }
            except Exception as e:
                failed_prompts.append(prompt)
                print(f"Error processing prompt: {str(e)}")
                return None

    # 创建所有任务
    tasks = [process_single_prompt(prompt) for prompt in prompt_list]
    
    # 执行并带进度条
    pbar = tqdm_asyncio(total=len(tasks), desc="Processing prompts")
    
    for future in asyncio.as_completed(tasks):
        result = await future
        if result:
            results.append(result)
        pbar.update()

    pbar.close()

    # 将结果转换为DataFrame并保存为Excel
    df = pd.DataFrame(results)
    df.to_csv(output_file_path, index=False)

    # 打印简要报告
    print(f"\nSuccess: {len(results)}")
    print(f"Failed: {len(failed_prompts)}")

    return failed_prompts


In [ ]:
failed_prompts = await process_prompts_async(
    prompt_list,
    output_file_path="raw_response_北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv"
)

Processing prompts: 100%|██████████| 5/5 [00:55<00:00, 11.11s/it]


Success: 5
Failed: 0


In [13]:
import pandas as pd
import re

# 读取原始Excel文件
input_path = "raw_response_北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.csv"
df = pd.read_csv(input_path)

# 定义一个函数来提取<CODE>标签中的内容
def extract_code(text):
    if not isinstance(text, str):
        return ""
    
    # 处理<\CODE>的错误格式，统一替换为</CODE>
    text = text.replace('<\\CODE>', '</CODE>')
    
    # 使用正则表达式提取所有<CODE>...</CODE>之间的内容
    code_blocks = re.findall(r'<CODE>(.*?)</CODE>', text, re.DOTALL)
    
    # 将所有代码块合并，用两个换行符分隔
    return '\n\n'.join(code_blocks).strip()

# 应用函数提取代码
df['code'] = df['response'].apply(extract_code)

# 保存到新的Excel文件
output_path = "code_北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx"
df.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到: {output_path}")

处理完成，结果已保存到: code_北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx


In [15]:
import pandas as pd
import nbformat as nbf

# 读取 Excel 文件
excel_file = "code_北京证券交易所上市公司持续监管指引第8号——股份减持和持股管理.xlsx"
df = pd.read_excel(excel_file)

# 初始化一个 Jupyter Notebook
notebook = nbf.v4.new_notebook()

# 遍历每一行数据
for _, row in df.iterrows():
    # 获取 MEU 编号
    prompt = row.get('prompt', '')
    code = row.get('code', '')

    # 从 prompt 提取 MEU 编号
    lines = prompt.split('\n')
    meu_line = next((line for line in lines if line.startswith("法律的最小可执行单元编号: ")), None)
    title = meu_line.split(": ")[1] if meu_line else "Unknown"

    # 添加 Markdown 单元格
    markdown_cell = nbf.v4.new_markdown_cell(f"# {title}\n\n{prompt}")
    notebook.cells.append(markdown_cell)

    # 添加代码单元格
    code_cell = nbf.v4.new_code_cell(code)
    notebook.cells.append(code_cell)

# 保存为 .ipynb 文件
output_file = "generated_notebook.ipynb"
with open(output_file, "w", encoding="utf-8") as f:
    nbf.write(notebook, f)

print(f"Notebook saved as {output_file}")

Notebook saved as generated_notebook.ipynb
